In [31]:
type 'a tree = F of 'a | N of 'a tree * 'a tree

type 'a tree = F of 'a | N of 'a tree * 'a tree


In [32]:
type 'a fp = { a : 'a array; mutable n: int } ;;

let make_fp p e = { a = Array.make p e; n = 0 } ;; 
let create () = make_fp 1000 (0,F("a")) ;; 
let pred i = (i - 1)/2
let g i = 2*i + 1
let d i = 2*i + 2
let swap fp i j =
    let tmp = fp.a.(i) in
        fp.a.(i) <- fp.a.(j);
        fp.a.(j) <- tmp ;;
let rec up fp i =
  let p = pred i in
  if i <> 0 && fst fp.a.(p) > fst fp.a.(i) then (
    swap fp i p;
    up fp p
    ) ;;
let rec down fp i =
    let get j = (if j < fp.n then fst fp.a.(j) else max_int), j in
    let m, j = min (get (2*i + 1)) (get (2*i + 2)) in
    if fst fp.a.(i) > m then (
    swap fp i j;
    down fp j 
    ) ;;
let add fp e =
    fp.a.(fp.n) <- e;
    up fp fp.n;
    fp.n <- fp.n + 1 ;;

let rec extract_min fp =
    swap fp 0 (fp.n - 1);
    fp.n <- fp.n - 1;
    down fp 0;
    fp.a.(fp.n)

let is_empty fp =
    fp.n = 0 ;;

let len_over_2 fp =
    fp.n >= 2 ;;

type 'a fp = { a : 'a array; mutable n : int; }


val make_fp : int -> 'a -> 'a fp = <fun>


val create : unit -> (int * string tree) fp = <fun>


val pred : int -> int = <fun>


val g : int -> int = <fun>


...

val down : (int * 'a) fp -> int -> unit = <fun>


val add : ('a * 'b) fp -> 'a * 'b -> unit = <fun>


val extract_min : (int * 'a) fp -> int * 'a = <fun>


val is_empty : 'a fp -> bool = <fun>


val len_over_2 : 'a fp -> bool = <fun>


In [33]:
let rec read arb lst =
  match arb with
  | F(chr) -> chr, lst
  | N(g, d) -> match lst with
                |0::t -> read g t
                |1::t -> read d t
                |_ -> failwith "liste non valide"

val read : 'a tree -> int list -> 'a * int list = <fun>


In [34]:
let rec decode arb lst =
  match (read arb lst) with
  |chr, [] -> chr::[]
  |chr, lst_end -> chr::(decode arb lst_end) ;;

val decode : 'a tree -> int list -> 'a list = <fun>


In [35]:
let to_huffman freq =
    let fp = create () in
    let rec fill freq=
      match freq with
      |[] -> ()
      |(f, chr)::t -> add fp (f, F(chr)) ; fill t in
    fill freq;
    while len_over_2 fp do
      let (f1, a1), (f2, a2) = extract_min fp, extract_min fp in
        add fp (f1 + f2, N(a1, a2))
    done;
    snd (extract_min fp) ;;

val to_huffman : (int * string) list -> string tree = <fun>


In [36]:
let arb = to_huffman [(20, "a"); (15, "b"); (7, "c"); (14, "d"); (44, "e")]

val arb : string tree = N (N (N (F "a", F "b"), N (F "d", F "c")), F "e")


In [37]:
let to_dict arb = 
  let dict = Hashtbl.create 100 in
  let rec aux arb dict code =
    match arb with
    |F(chr) -> Hashtbl.add dict chr (List.rev code)
    |N(g, d) -> aux g dict (0::code) ; aux d dict (1::code) 
  in
  aux arb dict [] ; dict ;;
  

val to_dict : 'a tree -> ('a, int list) Hashtbl.t = <fun>


In [38]:
let rec code lst dict =
    match lst with
    |[] -> []
    |h::t -> (Hashtbl.find dict h) @ code t dict

val code : 'a list -> ('a, 'b list) Hashtbl.t -> 'b list = <fun>


In [39]:
let dict = to_dict arb
let test = Hashtbl.find dict "b"

val dict : (string, int list) Hashtbl.t = <abstr>


val test : int list = [0; 0; 1]


In [44]:
let message = ["b"; "a"; "c"; "a"; "d"; "e"; "b"; "a"; "c"; "a"; "d"; "e"]
let encoded = code message dict
let encoded_err = encoded @ [0]

val message : string list =
  ["b"; "a"; "c"; "a"; "d"; "e"; "b"; "a"; "c"; "a"; "d"; "e"]


val encoded : int list =
  [0; 0; 1; 0; 0; 0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 1; 0; 0; 1; 0; 0; 0; 0; 1; 1;
   0; 0; 0; 0; 1; 0; 1]


val encoded_err : int list =
  [0; 0; 1; 0; 0; 0; 0; 1; 1; 0; 0; 0; 0; 1; 0; 1; 0; 0; 1; 0; 0; 0; 0; 1; 1;
   0; 0; 0; 0; 1; 0; 1; 0]


In [45]:
let decoded = decode arb encoded

val decoded : string list =
  ["b"; "a"; "c"; "a"; "d"; "e"; "b"; "a"; "c"; "a"; "d"; "e"]
